# Deep Learning Project - Orca Call Clustering 
Karlee Zammit - V00823093 

https://github.com/earthspecies/library/tree/main/orcas

https://debuggercafe.com/pytorch-pretrained-efficientnet-model-image-classification/

In [1]:
from __future__ import print_function, division
import os
import os.path, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
from torchvision.io import read_image
from torchvision import transforms, utils
from torchvision.datasets import ImageFolder
import torchvision.models as models
import librosa
import wave
from IPython.display import Audio
from skimage import io, transform
from PIL import Image, ImageOps
import umap
#import umap.plot
#from sklearn.datasets import load_digits
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.cluster import KMeans
!pip install torch torchvision efficientnet_pytorch==0.7.0
from efficientnet_pytorch import EfficientNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\kzammit\Miniconda3\envs\hotdog\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\kzammit\Miniconda3\envs\hotdog\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


ModuleNotFoundError: No module named 'numba'

# Functions

In [2]:
# Plot and save the waveform from the wave file 
def plot_waveform(idx, waveform, sr, title="Waveform"):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sr

    figure, axes = plt.subplots(num_channels, 1)
    axes.plot(time_axis, waveform[0], linewidth=1)
    axes.grid(True)
    figure.suptitle(title)
    #plt.show(block=False)
    #plt.savefig(r'C:\Users\kzammit\Repos\school\phys555\project-DL\orcas_classification\wavpngs\wav-' + str(idx) + '.png')
    plt.close()

In [3]:
# Plot and save the spectrogram from the wave file
#def create_spectrogram(idx, specgram, title=None, ylabel="freq_bin"):
def create_spectrogram(idx, specgram):
    #fig, axs = plt.subplots(1, 1)
    #axs.set_title(title or "Spectrogram (db)")
    #axs.set_ylabel(ylabel)
    #axs.set_xlabel("frame")
    #axs.set_ylim(0, 100)
    #print(specgram.shape)
    #imgf = np.flip(specgram, axis=0)
    transform=transforms.ToPILImage()
    #specgram=transform(specgram)
    specgram= transform(specgram)
    specgram = ImageOps.flip(specgram)
    specgram.save(r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\specs-db\spec-" + str(idx) + ".png")
    #display(specgram)
    
    #im = Image.fromarray(librosa.power_to_db(specgram))
    #if im.mode != 'RGB':
    #    im = im.convert('RGB')
    #im.save("your_file.jpeg")
    #im = axs.imshow(librosa.power_to_db(specgram), origin="lower", aspect="auto")
    # librosa... is a numpy array, save that to a png file instead 
    #fig.colorbar(im, ax=axs)
    #plt.show(block=False)
    #plt.savefig(r'C:\Users\kzammit\Repos\school\phys555\project-DL\orcas_classification\spectrograms\spec-' + str(idx) + '.png')
    #plt.close()

In [4]:
# Crop the spectrogram figures and save as pngs
def crop(path, dirs):
    for item in dirs:
        fullpath = os.path.join(path,item)
        if os.path.isfile(fullpath):
            im = Image.open(fullpath)
            f, e = os.path.splitext(fullpath)
            # last one does the height
            # left top right bottom
            imCrop = im.crop((80, 58, 475, 425)) #corrected
            #imCrop.save(f + '-crop.png', "PNG", quality=300)
            display(imCrop)

In [5]:
def preprocess():
    
    transform =  transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])
        
        #transforms.Normalize(mean=[0.13214], 
        #                     std=[0.04182]),]) # this is not the way to go 
    return transform

# Spectrogram Generation and Preprocessing

In [ ]:
# https://pytorch.org/audio/main/generated/torchaudio.transforms.AmplitudeToDB.html
# For each available wave file
for ii in range(0, 594):
    
    # In the audio folder 
    path = r'C:\Users\kzammit\Documents\PHYS555\orcas_classification\audio'
    
    # Get the file name and path
    file = '\\' + str(ii) + '.wav'
    fullfile = path + file

    # Set the sample to this file
    SAMPLE_WAV = fullfile

    # Load the wave with torchaudio 
    ORCA_WAVEFORM, SAMPLE_RATE = torchaudio.load(SAMPLE_WAV, normalize=True)
    transform = T.AmplitudeToDB(stype="amplitude", top_db=100)
    orca_waveform_db = transform(ORCA_WAVEFORM)

    # Plot the original waveform 
    #plot_waveform(idx=str(ii), waveform=ORCA_WAVEFORM, sr=SAMPLE_RATE, title="Original waveform")
    #plot_waveform(idx=str(ii), waveform=orca_waveform_db, sr=SAMPLE_RATE, title="Original waveform")
    #Audio(ORCA_WAVEFORM.numpy(), rate=SAMPLE_RATE)
    Audio(orca_waveform_db.numpy(), rate=SAMPLE_RATE)

    # Set the spectrogram parameters 
    n_fft = 2000
    win_length = 2000
    hop_length = 100

    # Define spectrogram settings using torch audio transforms  
    spectrogram = T.Spectrogram(n_fft=n_fft, win_length=win_length, hop_length=hop_length, center=True, pad_mode="reflect", power=2.0)
    
    # Calculate the spectrogram 
    #spec = spectrogram(ORCA_WAVEFORM)
    spec = spectrogram(orca_waveform_db)
    
    # Plot and save the spectrogram
    create_spectrogram(idx=str(ii), specgram=spec[0])

# Load in ORCA Data

In [6]:
class OrcaImageDataset(Dataset):
    
    def __init__(self, annotations_file, img_dir, transform, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        image = image.squeeze()
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [7]:
annotations_file = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\annotations-edited.csv"
#img_dir = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\specs"
img_dir = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\specs-db"
transforms_eff = preprocess()
orca_dataset = OrcaImageDataset(annotations_file, img_dir, transform=transforms_eff)
labels = orca_dataset.img_labels

In [9]:
xtrain_data, test_data, ytrain_labels, test_labels = train_test_split(
    orca_dataset, labels, test_size=0.2, random_state=21)

#  split a validation set from the training set
train_data, val_data, train_labels, val_labels = train_test_split(
    xtrain_data, ytrain_labels, test_size=0.2, random_state=22)

RuntimeError: Internal error.

In [ ]:
#scaler = MinMaxScaler()
#train_data_sc = scaler.fit_transform(train_data)
#df = pd.DataFrame(train_data)
#print(df[0][0].numpy())

In [ ]:
print('The number of samples for training is ' + str(len(train_data)))
print('The number of samples for validation is ' + str(len(val_data)))
print('The number of samples for testing is ' + str(len(test_data)))

In [ ]:
batch_size = 1
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
means = []
stds = []
for idex, data in enumerate(train_loader): 
    img, label = train_loader.dataset.__getitem__(idex)
    #display(img)
    mean = torch.mean(img)
    mean = float(mean)
    std = torch.std(img)
    means.append(mean)
    stds.append(std)

In [ ]:
#print(means)
meanf = np.mean(means)
print(meanf) 

stdf = np.std(means)
print(stdf) 

# Data Visualization

In [ ]:
# Crop the spectrograms and save as images 
path = r"C:\Users\kzammit\Documents\PHYS555\orcas_classification\example_crop"
dirs = os.listdir(path)
crop(path, dirs)

# the initial size of the image 

In [ ]:
print(orca_dataset.img_labels)

In [ ]:
print('The shape of tensor for 60th image in train dataset: ', orca_dataset[60][0].shape)
print(orca_dataset)

In [ ]:
print(orca_dataset[60][0])

In [ ]:
transform = transforms.ToPILImage()
plt.figure(figsize=(16, 6))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    image, _ = train_loader.dataset.__getitem__(i)
    #print(image.shape)
    image = transform(image)
    plt.imshow(image)
    plt.axis('off');
    #print(image.size)

# Models

In [ ]:
def train_test_model(model, train_loader, val_loader, n_epochs, optimizer):
    criterion = nn.MSELoss()
    optimizer = optimizer
    model = model.train()
    train_losses = []
    val_losses = []
    
    for epoch in range(n_epochs):
        train_batch_losses = []
        val_batch_losses = []
        for data in train_loader:
            img, _ = data
            #img = img.squeeze()
            #print(img.shape)
            img = img.to(device)
            #img = img.reshape(-1, 28*28)
            
            #print(img.shape)
            output = model(img)
            loss = criterion(output, img.data)
            #************************ backward *************************
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_batch_losses.append(loss.item()) 
            
        model = model.eval()
        for data in val_loader:
            output = model(img)
            loss = criterion(output, img.data)
            val_batch_losses.append(loss.item())
            
        train_loss = np.mean(train_batch_losses)
        val_loss = np.mean(val_batch_losses)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
            
        # ***************************** log ***************************
        if epoch % 10 == 0:
            print(f"epoch [{epoch + 1}/{n_epochs}], Train loss:{train_loss: .4f} Valid:{val_loss: .4f}")
            
    ax = plt.figure().gca()
    ax.plot(train_losses)
    ax.plot(val_losses)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'])
    plt.title('Loss monitoring')
    plt.show()
    
    return model.eval()

# Autoencoder

In [ ]:
# Creating a DeepAutoencoder class
class DeepAutoencoder(torch.nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            #torch.nn.Linear(28 * 28, 256),
            torch.nn.Linear(224, 224),
            torch.nn.ReLU(),
            torch.nn.Linear(224, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 10)
        )
          
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(10, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 224),
            torch.nn.ReLU(),
            torch.nn.Linear(224, 224),
            torch.nn.Sigmoid()
        )
  
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# Instantiating the model and hyperparameters
model = DeepAutoencoder().to(device)
print(model)
n_epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
linear_model = train_test_model(model, train_loader, val_loader, n_epochs, optimizer)

## CNN Autoencoder

In [ ]:
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        ## encoder layers ##
        
        self.encoder = nn.Sequential( 
            nn.Conv2d(1,16,3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(16,4,3, padding=1),
            nn.ReLU(), 
            nn.MaxPool2d(2,2)
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(4,16,2, stride=2), 
            nn.ReLU(), 
            nn.ConvTranspose2d(16,1,2, stride=2),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
# initialize the NN
model = ConvAutoencoder().to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
n_epochs=100
cnn_model = train_test_model(model, train_loader, val_loader, n_epochs, optimizer)

# Exploring the Output

## Linear 

In [ ]:
transform = transforms.ToPILImage()
fig1 = plt.figure(figsize=(16, 6))
fig2 = plt.figure(figsize=(16, 6))
fig3 = plt.figure(figsize=(16, 6))
for idex, data in enumerate(train_loader):
    if idex<10:
        
        img, _ = data
        img = img.to(device)
        
        imgs = img.squeeze()
        imgs = transform(imgs)
        
        lspace = linear_model.encoder(img)
        lspaces = lspace.squeeze()
        image_l = transform(lspaces)
        
        output = linear_model(img)
        output_sq = output.squeeze()
        image_o = transform(output_sq)
        
        plt.figure(1)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(imgs)
        plt.title('Original Spectrogram ' + str(idex))
        plt.axis('off');
        
        plt.figure(2)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(image_l)
        plt.title('Latent Space ' + str(idex))
        plt.axis('off');
        
        plt.figure(3)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(image_o)
        plt.title('Reconstruction ' + str(idex))
        plt.axis('off');

## CNN

In [ ]:
lspacess = []
labels = []
transform = transforms.ToPILImage()
fig1 = plt.figure(figsize=(16, 6))
fig2 = plt.figure(figsize=(16, 6))
fig3 = plt.figure(figsize=(16, 6))
for idex, data in enumerate(train_loader):
    
    # plotting
    if idex<10:
        
        img, _ = data
        img = img.to(device)

        imgs = img.squeeze()
        imgs = transform(imgs)

        lspace = cnn_model.encoder(img)
        lspaces = lspace.squeeze()
        image_l = transform(lspaces)

        output = cnn_model(img)
        output_sq = output.squeeze()
        image_o = transform(output_sq)

        plt.figure(1)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(imgs)
        plt.title('Original Spectrogram ' + str(idex))
        plt.axis('off');

        plt.figure(2)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(image_l)
        plt.title('Latent Space ' + str(idex))
        plt.axis('off');

        plt.figure(3)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(image_o)
        plt.title('Reconstruction ' + str(idex))
        plt.axis('off');

In [ ]:
#conv_model2 = CNN_autoencoder2().to(device)
#num_epochs = 100
#optimizer = torch.optim.Adam(conv_model2.parameters(), lr=1e-3)
lspaces = []
labels = []
for data in train_loader:
    img, _ = data
    img = img.to(device)
    lspace = cnn_model.encoder(img)
    output = cnn_model(img)
    #print(lspace.shape)
    lspacef = torch.flatten(lspace, start_dim=1, end_dim=-1)
    #print(lspacef.shape)
    lspacen = lspacef.to('cpu')
    lspacen = lspacen.detach().numpy()
    lspaces.append(lspacen)
    if data[1][0]=='call':
        labels.append(1)
    else:
        labels.append(0)

In [ ]:
edited = []
for ii in range(0, len(lspaces)):
    edited.append(lspaces[ii][0])

In [ ]:
df = pd.DataFrame(edited)

In [ ]:
reducer = umap.UMAP()
embedding = reducer.fit_transform(df)
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=labels, alpha=0.5)
plt.legend(['call','no call']) # FIGURE THIS OUT
plt.title('UMAP Projection of the Orca Dataset from CNN Output', fontsize=24);

In [ ]:
hover_data = pd.DataFrame({'index':np.arange(380),
                           'label':labels})
hover_data['item'] = hover_data.label.map(
    {
        0:'no call',
        1:'call'
    }
)

In [ ]:
#https://umap-learn.readthedocs.io/en/0.4dev/plotting.html
umap.plot.output_notebook()
mapper = umap.UMAP().fit(df)

In [ ]:
p = umap.plot.interactive(mapper, labels=labels, hover_data=hover_data, point_size=8, theme='blue')
umap.plot.show(p)

In [ ]:
kmeans = KMeans(n_clusters=2, init="random", n_init=10, max_iter=300, random_state=42)
label_kmean = kmeans.fit_predict(df)

df['kmeanc'] = label_kmean

plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=label_kmean, alpha=0.5)
plt.title('KMean Cluster Colouring on UMAP Projection')

In [ ]:
transform = transforms.ToPILImage()
fig1 = plt.figure(figsize=(16, 6))
fig2 = plt.figure(figsize=(16, 6))
for idex, data in enumerate(train_loader):
    if idex<10:
        
        img, _ = data
        img = img.to(device)
        
        imgs = img.squeeze()
        imgs = transform(imgs)
        
        lspace = cnn_model.encoder(img)
        lspaces = lspace.squeeze()
        image_l = transform(lspaces)
        
        output = cnn_model(img)
        output_sq = output.squeeze()
        image_o = transform(output_sq)
        
        plt.figure(1)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(image_l)
        plt.title('kmean label ' + str(label_kmean[idex]))
        plt.axis('off');
        
        plt.figure(2)
        plt.subplot(2, 5, idex + 1)
        plt.imshow(imgs)
        plt.title('kmean label ' + str(label_kmean[idex]))
        plt.axis('off');
        

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(df) 

plt.scatter(
    z[:, 0],
    z[:, 1],
    c=labels, alpha=0.5)
plt.title('T-SNE Clustering')

In [ ]:
plt.scatter(
    z[:, 0],
    z[:, 1],
    c=label_kmean, alpha=0.5)
plt.title('TSNE Clustering with K-Mean Cluster Colouring')

# stuff that doesn't work

In [ ]:
# Fake it till u make it bb
# https://towardsdatascience.com/transfer-learning-on-greyscale-images-how-to-fine-tune-pretrained-models-on-black-and-white-9a5150755c7a
def expand_greyscale_image_channels(grey_image):
    grey_image = grey_image.to('cpu')
    grey_image_arr = np.array(grey_image)
    grey_image_arr = np.expand_dims(grey_image_arr, -1)
    grey_image_arr_3_channel = grey_image_arr.repeat(3, axis=-1)
    grey_image_arr_3_channel = torch.from_numpy(grey_image_arr_3_channel)
    grey_image_arr_3_channel = grey_image_arr_3_channel.to(device)
    
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = torch.squeeze(grey_image_arr_3_channel)
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = grey_image_arr_3_channel.T
    print(grey_image_arr_3_channel.shape)
    grey_image_arr_3_channel = grey_image_arr_3_channel.unsqueeze(0)
    print(grey_image_arr_3_channel.shape)
    return grey_image_arr_3_channel

In [ ]:
class PreT_autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = EfficientNet.from_pretrained('efficientnet-b0', include_top=False)

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(1280, 512, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 224, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(224, 1, kernel_size=4, stride=3, padding=1),
            nn.Upsample(1, 1, 2, 2)
        )
        

    def forward(self, x):
        x = expand_greyscale_image_channels(x)
        x = self.encoder.extract_features(x)
        #print(x.shape)
        lspace = x
        for l in self.decoder:
            x = l(x)
            #print(x.shape)
        print(x.shape)
        return x, lspace

In [ ]:
preT_model = PreT_autoencoder().to(device)
num_epochs = 100
#output, lspace = train_model(conv_model, train_loader, num_epochs)
optimizer = torch.optim.Adam(preT_model.parameters(), lr=1e-3)
model, output, lspace = train_test_model(preT_model, train_loader, val_loader, num_epochs, optimizer)

In [ ]:
def train_test_model(model, train_loader, val_loader, n_epochs, optimizer):
    #transform = transforms.ToPILImage()
    criterion = nn.MSELoss()
    optimizer = optimizer
    model = model.train()
    train_losses = []
    val_losses = []
    
    for epoch in range(n_epochs):
        train_batch_losses = []
        val_batch_losses = []
        for data in train_loader:
            img, _ = data
            img    = img.to(device)
            output, lspace = model(img)
            loss = criterion(output, img.data)
            #************************ backward *************************
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_batch_losses.append(loss.item()) 
            
        model = model.eval()
        for data in val_loader:
            output, lspace = model(img)
            loss = criterion(output, img.data)
            val_batch_losses.append(loss.item())
            
        train_loss = np.mean(train_batch_losses)
        val_loss = np.mean(val_batch_losses)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
            
        # ***************************** log ***************************
        if epoch % 100 == 0:
            print(f"epoch [{epoch + 1}/{num_epochs}], Train loss:{train_loss: .4f} Valid:{val_loss: .4f}")
    
    ax = plt.figure().gca()
    ax.plot(train_losses)
    ax.plot(val_losses)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'val'])
    plt.title('Loss monitoring')
    plt.show()
    
    return model.eval(), output, lspace

In [ ]:
# https://medium.com/thecyphy/train-cnn-model-with-pytorch-21dafb918f48
class CNN_autoencoder2(nn.Module):
    def __init__(self):
        super().__init__() 
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(401408,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,40)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(40, 512),
            nn.ReLU(), 
            nn.Linear(512, 1024), 
            nn.ReLU(), 
            nn.Linear(1024, 401408),
            nn.Unflatten(1, torch.Size([128, 56, 56])), 
            
            nn.MaxUnpool2d(2,2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 128, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(), 
            
            nn.MaxUnpool2d(2,2), 
            nn.ReLU(), 
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=1, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, padding=1)
        )
           
    def forward(self, x):
        for l in 
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
conv_model2 = CNN_autoencoder2().to(device)
num_epochs = 100
optimizer = torch.optim.Adam(conv_model2.parameters(), lr=1e-3)
conv_model = train_test_model2(conv_model2, train_loader, val_loader, num_epochs, optimizer)